<a href="https://colab.research.google.com/github/DavidHospinal/MPREAC-OCEANAPP/blob/main/Final_VI_Deploy__aws_OceanApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo

YOLO es una familia de modelos de detección de objetos a escala compuesta entrenados en COCO dataset, e incluye una funcionalidad simple para Test Time Augmentation (TTA), model ensembling, hyperparameter evolution, and export to ONNX, CoreML and TFLite.


## Gradio Inferencia

![](https://i.ibb.co/982NS6m/header.png)

  Este Notebook se acelera opcionalmente con un entorno de ejecución de GPU


----------------------------------------------------------------------

 YOLOv5 Gradio demo

*Author: Ultralytics LLC and Gradio*

# Código

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 10.2 MB 54.5 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 140 kB 76.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [ ]:
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt gradio # install dependencies

     |████████████████████████████████| 11.6 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 74.0 MB/s 
     |████████████████████████████████| 127 kB 48.9 MB/s 
     |████████████████████████████████| 106 kB 67.2 MB/s 
     |████████████████████████████████| 2.3 MB 44.2 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 278 kB 78.1 MB/s 
     |████████████████████████████████| 213 kB 78.2 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 1.6 MB 60.7 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 80 kB 10.2 MB/s 
     |████████████████████████████████| 68 kB 6.7 MB/s 
     |█████████████████████████

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Ingresar el codigo que esta en Aplicativo_OceanApp --> TRAIN_GOOGLE_COLAB --> awscli

In [ ]:
import gradio as gr
import pandas as pd
import torch
import logging
import json
import re
import os #Para las variables de entorno
#-------Para conectar con aws--------
import boto3
from botocore.exceptions import NoCredentialsError
import tempfile
import io
from PIL import Image
from pandas import json_normalize

In [ ]:
!export AWS_CONFIG_FILE=/content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_CONFIG_FILE'] = path
print(os.environ['AWS_CONFIG_FILE'])

/content/drive/My Drive/config/awscli.ini


In [ ]:
#Imagen temporal guardada en upload_file
def tempFileJSON(img_file):
  temp = tempfile.NamedTemporaryFile(mode="wb")
  with temp as jpg:
    jpg.write(img_file)
    print(jpg.name)
    uf = upload_file(jpg.name)
    return uf

# Envio de imagenes a S3
def upload_file(file_name, bucket=None, object_name=None):
  """Upload a file to an S3 bucket

  :param file_name: File to upload
  :param bucket: Bucket to upload to
  :param object_name: S3 object name. If not specified then file_name is used
  :return: Json if file was uploaded, else False
  """
  # If S3 object_name was not specified, use file_name
  if object_name is None:
    object_name = os.path.basename(file_name+".jpg")
  if bucket is None:
    bucket = 'oceanapp'
  #Se esta eliminando el aws client
  s3_client = boto3.client('s3') #comentar si no funciona aws---------
  aws_region = boto3.session.Session().region_name #comentar si no funciona aws---------
  # Upload the file
  try:
    with open(file_name, "rb") as f:
      response = s3_client.upload_fileobj(f, bucket, object_name) #comentar si no funciona aws---------
      s3_url = f"https://{bucket}.s3.amazonaws.com/{object_name}" #comentar si no funciona aws---------
      stado = '"url_details":[{"statusCode":200, "s3_url":"'+s3_url+'"}]'
    print(s3_url)
  except FileNotFoundError:
    print("The file was not found")
    return False
  except NoCredentialsError as e:
    logging.error(e)
    return False
  return stado

In [ ]:
def removeStr(string): 
    return string.replace(" ", "")

def qtyEspecies(datax, datay, resImg):
  numLobos = 0
  numPelicanos = 0
  dfEspecies = pd.DataFrame(datax)
  datay = str(datay)
  for i in range(0, dfEspecies['name'].size):
    if(dfEspecies['class'][i] == 0):
      numLobos= numLobos + 1
    if(dfEspecies['class'][i] == 1):
      numPelicanos= numPelicanos + 1
  strlista = '"detail":[{"quantity":"'+str(numLobos)+'","description":"Lobo marino"},{"quantity":"'+str(numPelicanos)+'","description":"Pelicano"}]'
  data = '{"image":"'+str(removeStr(datay[0:9]))+'","size":"'+str(removeStr(datay[11:18]))+'",'+strlista+','+resImg+'}'
  json_data = json.loads(data)
  return json_data

def arrayListax(json_data):
  dict = json_data
  df2 = json_normalize(dict['detail']) 
  #df = pd.DataFrame(df2,columns=['Cantidad','Especie'])
  return df2
  

In [ ]:
#------------ Interface-------------
# Images
torch.hub.download_url_to_file('https://i.pinimg.com/564x/18/0b/00/180b00e454362ff5caabe87d9a763a6f.jpg', 'ejemplo1.jpg')
torch.hub.download_url_to_file('https://i.pinimg.com/564x/3b/2f/d4/3b2fd4b6881b64429f208c5f32e5e4be.jpg', 'ejemplo2.jpg')

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True, autoshape=True)  # local model  o google colab

def yolo(size, iou, conf, im):
    try:
      '''Wrapper fn for gradio'''
      # gain
      g = (int(size) / max(im.size))  
      # resize
      #if(max(im.size)>900 and g>0.4):
        #g=0.3
      #if(max(im.size)>2000 and g>0.4):
        #g=0.1
      im = im.resize((int(x * g) for x in im.size), Image.ANTIALIAS)  

      model.iou = iou
      
      model.conf = conf
      # inference 
      results2 = model(im)  
      # updates results.imgs with boxes and labels
      results2.render() 
      #results5=results2.xyxy[0]  
      print(results2) 
      results5=results2.pandas().xyxy[0].sort_values('name')

      print('/////////////////////////////////////')
      results3 = str(results2)
      #Transformando la img en bytes
      pil_im = Image.fromarray(results2.ims[0])
      b = io.BytesIO()
      pil_im.save(b, 'jpeg')
      im_bytes = b.getvalue()
      fileImg = tempFileJSON(im_bytes)
      #Enviando la informacion al contador de especies
      results6=qtyEspecies(results5,results2,fileImg)
      print(results6)
      lista2 = arrayListax(results6)
      print(lista2)
      #lista = listJSON(results3[0:9], results3[11:18] ,results3[19:21],results3[22:32], results3[34:36], results3[36:45], fileImg)
      #lista2 = arrayLista(results3[19:21],results3[22:32], results3[34:36], results3[37:45])
      return Image.fromarray(results2.ims[0]), lista2, results6
    except Exception as e:
      logging.error(e, exc_info=True)

#------------ Interface-------------

in1 = gr.inputs.Radio(['640', '1280'], label="Tamaño de la imagen", default='640', type='value')
in2 = gr.inputs.Slider(minimum=0, maximum=1, step=0.05, default=0.45, label='NMS IoU threshold')
in3 = gr.inputs.Slider(minimum=0, maximum=1, step=0.05, default=0.50, label='Umbral o  threshold')
in4 = gr.inputs.Image(type='pil', label="Original Image")

out2 = gr.outputs.Image(type="pil", label="Identificación con Yolov5")
out3 = gr.outputs.Dataframe(type="pandas", label="Descripción", headers=['Cantidad','Especie'])  #El type no es necesario colocarlo en HugginFace
out4 = gr.outputs.JSON(label="JSON")

#-------------- Text-----
title = 'OceanApp'
description = """
<p>
<center>
<p>Sistema para el reconocimiento de las especies en la pesca acompañante de cerco, utilizando redes neuronales convolucionales para una empresa del sector pesquero en los puertos de callao y paracas.</p>
<p><b>Nota</b>: Este modelo solo acepta imagenes de <b>Lobos marinos</b> o <b>Pelicanos</b> proporcionados por empresas peruanas.</p>
<center>
<img src="https://i.pinimg.com/564x/3e/b8/f7/3eb8f7c348dffd7b3dffcafe81fbf2a6.jpg" alt="logo" width="250"/>
</center>
</center>
</p>
"""
article ="<p style='text-align: center'><a href='' target='_blank'>Para mas info, clik para ir al white paper</a></p><p style='text-align: center'><a href='https://colab.research.google.com/drive/1j0T8gdLIa0X8fzkIgFpXDoU27BF49RUz?usp=sharing' target='_blank'>Google Colab Demo</a></p><p style='text-align: center'><a href='https://github.com/MssLune/OceanApp-Model' target='_blank'>Repo Github</a></p></center></p>"
          
examples = [['640',0.45, 0.75,'ejemplo1.jpg'], ['640',0.45, 0.75,'ejemplo2.jpg']]

iface = gr.Interface(yolo, inputs=[in1, in2, in3, in4], outputs=[out2,out3,out4], title=title, description=description, article=article, examples=examples,theme="huggingface", analytics_enabled=False).launch(
    debug=True, share=True)

iface.launch()

  0%|          | 0.00/36.7k [00:00<?, ?B/s]

  0%|          | 0.00/26.7k [00:00<?, ?B/s]

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-12-5 Python-3.8.15 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f26831844752b71c.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


image 1/1: 426x640 1 Pelicano
Speed: 3.9ms pre-process, 262.5ms inference, 0.8ms NMS per image at shape (1, 3, 448, 640)
/////////////////////////////////////
/tmp/tmpjd4f2k07
https://oceanapp.s3.amazonaws.com/aaa
{'image': 'image1/1', 'size': '426x640', 'detail': [{'quantity': '0', 'description': 'Lobo marino'}, {'quantity': '1', 'description': 'Pelicano'}], 'url_details': [{'statusCode': 200, 's3_url': 'https://oceanapp.s3.amazonaws.com/aaa'}]}
  quantity  description
0        0  Lobo marino
1        1     Pelicano
Keyboard interruption in main thread... closing server.


AttributeError: ignored

For YOLOv5 PyTorch Hub inference with **PIL**, **OpenCV**, **Numpy** or **PyTorch** inputs please see the full [YOLOv5 PyTorch Hub Tutorial](https://github.com/ultralytics/yolov5/issues/36).


## Citation

[![DOI](https://zenodo.org/badge/264818686.svg)](https://zenodo.org/badge/latestdoi/264818686)